<a href="https://colab.research.google.com/github/EvgenyEsin/Spark_Apache/blob/main/Spark_Apache_sem_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ к семинару 5

создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=0249b41ef2b8e7ee505ce171db850d2f45497877fbc554b851038bf7baef4435
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

In [3]:
# Создание Spark сессии
spark = SparkSession.builder.appName("Homework5").getOrCreate()

In [4]:
# Загрузка данных
df = spark.read.option("header",True).csv("data.csv")
print("Исходный датасет:")
df.show()

Исходный датасет:
+--------------------+--------------------+--------------------+-----+------+
|               title|              author|               genre|sales|  year|
+--------------------+--------------------+--------------------+-----+------+
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|
|"""The Lord of th...|  ""J.R.R. Tolkien""|         ""Fantasy""| 3000| 1954"|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|
|"""The Catcher in...|   ""J.D. Salinger""|           ""Novel""| 2000| 1951"|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|
+--------------------+--------------------+--------------------+-----+------+



In [5]:
# Фильтрация данных, чтобы оставить только книги, продажи которых превышают 3000 экземпляров
filtered_df = df.filter(df.sales > 3000)
print("Фильтрация по продажам (более 3000 экземпляров):")
filtered_df.show()

Фильтрация по продажам (более 3000 экземпляров):
+--------------------+--------------------+--------------------+-----+------+
|               title|              author|               genre|sales|  year|
+--------------------+--------------------+--------------------+-----+------+
|           """1984""|   ""George Orwell""| ""Science Fiction""| 5000| 1949"|
|"""To Kill a Mock...|      ""Harper Lee""| ""Southern Gothic""| 4000| 1960"|
|"""The Great Gats...| ""F. Scott Fitzg...|           ""Novel""| 4500| 1925"|
+--------------------+--------------------+--------------------+-----+------+



In [6]:
# Группировка данных по жанру и вычисление общего объема продаж для каждого жанра
window_spec = Window.partitionBy("genre")
sum_sales_by_genre = fn.sum("sales").over(window_spec)
sum_sales_by_genre_df = df.withColumn("sum_sales_by_genre", sum_sales_by_genre)

In [7]:
# Сортировка данных по общему объему продаж в порядке убывания и вывод результатов:
print("Общие объемы продаж по жанрам в порядке убывания:")
sum_sales_by_genre_df.select(["genre", "sum_sales_by_genre"]).distinct() \
    .orderBy(sum_sales_by_genre_df.sum_sales_by_genre.desc()).show()

Общие объемы продаж по жанрам в порядке убывания:
+--------------------+------------------+
|               genre|sum_sales_by_genre|
+--------------------+------------------+
|           ""Novel""|            6500.0|
| ""Science Fiction""|            5000.0|
| ""Southern Gothic""|            4000.0|
|         ""Fantasy""|            3000.0|
+--------------------+------------------+

